In [3]:
import PyQt5

In [4]:
%matplotlib qt5

# 调入必备库

In [1]:
import time
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)
torch.manual_seed(42)

## 所解决的方程
$u_t=vu_{xx}-\alpha  u u_x$  $x \in [-1,1]$  $t \in [-1,1]$

$ u(t,0)=a $

$ u(t,1)=b $

$ u(0,x)=g(x)$

$g(x)=(b-a)x+a$

# 搭建基础网络结构

In [5]:
class zsrDGM_net(nn.Module):
    def __init__(self,numl,numn):
        # numl是有多少层隐藏层
        # numn是每层的神经元数量
        super(zsrDGM_net, self).__init__()
        self.input_layer = nn.Linear(2, numn)#前面的数字代表几个输入
        self.hidden_layers = nn.ModuleList([nn.Linear(numn, numn) for i in range(numl)])
        self.output_layer = nn.Linear(numn, 1)
    def forward(self, x):
        o = self.act(self.input_layer(x))
        for i, li in enumerate(self.hidden_layers):
            o = self.act(li(o))        
        out = self.output_layer(o)        
        return out
    def act(self, x):
        return x * torch.tanh(x)

# 建立equation

In [6]:
class burgers_PDE():
    def __init__(self,net,v,alpha,a,b):
        self.net=net
        self.v=v
        self.alpha=alpha
        self.a=a
        self.b=b
    def sample(self,size=2**8):        
        x = torch.cat((torch.rand([N1*size, 1]), (torch.rand([N1*size, 1]))),dim=1)#在范围内建立x采样点
        x_init = torch.full([N2*size, 1], 0) + torch.rand([N2*size, 1]) 
        x_initial = torch.cat((torch.zeros(N2*size, 1), x_init), dim=1)#建立初始条件采样
        x_boundary_left = torch.cat((torch.rand([N3*size, 1]), torch.full([N3*size, 1], 0)), dim=1)
        x_boundary_right = torch.cat((torch.rand([N3*size, 1]), torch.full([N3*size, 1],1)), dim=1)
        return x, x_initial, x_init, x_boundary_left, x_boundary_right
    def loss_func(self,size=2**8):
        x_train, x_initial, x_init, x_boundary_left, x_boundary_right = self.sample(size=size)
        x = Variable(x_train, requires_grad=True)
        uy=net(x)
        d = torch.autograd.grad(net(x), x, grad_outputs=torch.ones_like(net(x)), create_graph=True)
        dt = d[0][:, 0].unsqueeze(-1)
        dx = d[0][:, 1].unsqueeze(-1)
        dxx = torch.autograd.grad(dx, x, grad_outputs=torch.ones_like(dx), create_graph=True)[0][:, 1].unsqueeze(-1)
        loss_fn = nn.MSELoss(reduction='mean')
        loss1 = loss_fn(dt, self.v*dxx-self.alpha*uy*dx)
        loss2 = loss_fn(net(x_initial), torch.zeros([N2*size,1])+(self.b-self.a)*x_init+self.a)
        loss3 = loss_fn(net(x_boundary_left), torch.zeros([N3*size,1])+torch.full([N3*size, 1], self.a))
        loss4 = loss_fn(net(x_boundary_right),torch.zeros([N3*size,1])+torch.full([N3*size, 1], self.b))
        #print('loss1',loss1)
        #print('loss2',loss2)
        #print('loss3',loss3)
        #print('loss4',loss4)
        loss = loss1 + loss2 + loss3 + loss4
        #print(loss1,loss2,loss3,loss4)
        return loss

# 训练设置

In [7]:
class Train():
    def __init__(self, net, eq, BATCH_SIZE):
        self.errors = []
        self.BATCH_SIZE = BATCH_SIZE
        self.net = net
        self.model = eq
    def train(self, epoch, lr):
        optimizer = optim.Adam(self.net.parameters(), lr)
        avg_loss = 0
        for e in range(epoch):
            optimizer.zero_grad()
            loss = self.model.loss_func(self.BATCH_SIZE)
            avg_loss = avg_loss + float(loss.item())
            loss.backward()
            optimizer.step()
            if e % 100 == 99:
                loss = avg_loss/100
                print("Epoch {} - lr {} -  loss: {}".format(e, lr, loss))
                avg_loss = 0
                error = self.model.loss_func(2**6)
                self.errors.append(error.detach())
    def get_errors(self):
        return self.errors

# 输入参数，开始训练

## 第一组参数

In [15]:
net = zsrDGM_net(6, 200)
v=0.01
alpha=0.95
a=0.9
b=-0.9
N1=1
N2=1
N3=3
equation = burgers_PDE(net,v,alpha,a,b)
train = Train(net, equation, BATCH_SIZE=2**8)
train.train(epoch=1500, lr=0.0001)
torch.save(net, 'model1.pkl')
errors = train.get_errors()

Epoch 99 - lr 0.0001 -  loss: 1.8800814354419708
Epoch 199 - lr 0.0001 -  loss: 1.2312651568651198
Epoch 299 - lr 0.0001 -  loss: 0.8064983975887299
Epoch 399 - lr 0.0001 -  loss: 0.7586843132972717
Epoch 499 - lr 0.0001 -  loss: 0.6053863170742989
Epoch 599 - lr 0.0001 -  loss: 0.04627862367779016
Epoch 699 - lr 0.0001 -  loss: 0.02226265612989664
Epoch 799 - lr 0.0001 -  loss: 0.013152490099892021
Epoch 899 - lr 0.0001 -  loss: 0.012029041331261396
Epoch 999 - lr 0.0001 -  loss: 0.01598816974554211
Epoch 1099 - lr 0.0001 -  loss: 0.013852900685742497
Epoch 1199 - lr 0.0001 -  loss: 0.008956187777221203
Epoch 1299 - lr 0.0001 -  loss: 0.007022836618125439
Epoch 1399 - lr 0.0001 -  loss: 0.005936511610634625
Epoch 1499 - lr 0.0001 -  loss: 0.005863192048855126


In [16]:
fig = plt.figure()
plt.plot(np.log(errors), '-b', label='Errors')
plt.title('Training Loss', fontsize=10)
plt.savefig('err1.jpg')
plt.close(fig)

In [8]:
net=torch.load('model1.pkl')
x_1 = torch.tensor([[i/1000] for i in range(0,1000)])
x_11 = torch.cat((torch.full([1000, 1],1), x_1), dim=1)
ysolve=net(x_11)
ysolve=ysolve.detach().numpy()
x_range=[i/1000 for i in range(0,1000)]
fig=plt.figure(dpi=60)
plt.plot(x_range,(ysolve))
plt.xlim(0,1)
plt.ylim(-1,1)
my_x_ticks = np.arange(0, 1.01, 0.2)
my_y_ticks = np.arange(-1, 1.1, 0.5)
plt.yticks(my_y_ticks)
plt.xticks(my_x_ticks)
plt.xlabel('x')
plt.ylabel('u')
plt.savefig('1.jpg')
plt.show()

In [18]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
net=torch.load('model1.pkl')
fig, ax = plt.subplots()

# 定义存储数据的列表
xdata = []
ydata = []

# 接收line2D对象
line, = plt.plot(xdata, ydata, 'b')
xdata=[i/100 for i in range(0,100)]

# 定义更新函数
def update(frames):
    x_1 = torch.tensor([[i/100] for i in range(0,100)])
    x_11 = torch.cat((torch.full([100, 1],frames), x_1), dim=1)
    ysolve1=net(x_11)
    ysolve=ysolve1.detach().numpy()
    ydata=[ysolve]
    # print(frame_ID)
    line.set_data(xdata, ydata)
    return line,


def init_figure():
    ax.set_xlim(0, 1)
    ax.set_ylim(-1, 1)
    ax.set_xlabel('x')
    ax.set_ylabel('u')


# 调用生成动画的函数生成动图
ani = animation.FuncAnimation(
    fig=fig,
    func=update,
    frames=np.linspace(0, 1, 100),    # [1, 2, 3]
    init_func=init_figure,
    interval=1,   #  每隔多少时间生成一帧图像，单位是ms
    repeat=True,   # 设置不重复，但是还是重复较好
)

plt.show()   # 如果要保存视频和gif就不要show()

ani.save('ani_1.gif', writer='pillow')
#ani.save('ani_1.mp4', writer='ffmpeg')  # 注意，pillow现在似乎不能报错为mp4格式了，可以使用ffmpeg

## 第二组参数

In [11]:
net = zsrDGM_net(6, 200)
v=0.02
alpha=0.95
a=0.9
b=-0.9
N1=1
N2=1
N3=2
equation = burgers_PDE(net,v,alpha,a,b)
train = Train(net, equation, BATCH_SIZE=2**8)
train.train(epoch=1500, lr=0.0001)
torch.save(net, 'model2.pkl')
errors = train.get_errors()

Epoch 99 - lr 0.0001 -  loss: 1.8541904497146606
Epoch 199 - lr 0.0001 -  loss: 1.041292937397957
Epoch 299 - lr 0.0001 -  loss: 0.8366931486129761
Epoch 399 - lr 0.0001 -  loss: 0.7926911145448685
Epoch 499 - lr 0.0001 -  loss: 0.19063443621620535
Epoch 599 - lr 0.0001 -  loss: 0.008141566663980485
Epoch 699 - lr 0.0001 -  loss: 0.004598336250055581
Epoch 799 - lr 0.0001 -  loss: 0.0037454722146503627
Epoch 899 - lr 0.0001 -  loss: 0.003409901645500213
Epoch 999 - lr 0.0001 -  loss: 0.004153324826620519
Epoch 1099 - lr 0.0001 -  loss: 0.0029859915538690985
Epoch 1199 - lr 0.0001 -  loss: 0.003740675845183432
Epoch 1299 - lr 0.0001 -  loss: 0.0024812370073050262
Epoch 1399 - lr 0.0001 -  loss: 0.0023978420719504355
Epoch 1499 - lr 0.0001 -  loss: 0.002222717055119574


In [12]:
fig = plt.figure()
plt.plot(np.log(errors), '-b', label='Errors')
plt.title('Training Loss', fontsize=10)
plt.savefig('err2.jpg')
plt.close(fig)

In [13]:
net=torch.load('model2.pkl')
x_1 = torch.tensor([[i/1000] for i in range(0,1000)])
x_11 = torch.cat((torch.full([1000, 1],1), x_1), dim=1)
ysolve=net(x_11)
ysolve=ysolve.detach().numpy()
x_range=[i/1000 for i in range(0,1000)]
fig=plt.figure(dpi=60)
plt.plot(x_range,(ysolve))
plt.xlim(0,1)
plt.ylim(-1,1)
my_x_ticks = np.arange(0, 1.01, 0.2)
my_y_ticks = np.arange(-1, 1.1, 0.5)
plt.yticks(my_y_ticks)
plt.xticks(my_x_ticks)
plt.xlabel('x')
plt.ylabel('u')
plt.savefig('2.jpg')
plt.show()

In [14]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
net=torch.load('model2.pkl')
fig, ax = plt.subplots()

# 定义存储数据的列表
xdata = []
ydata = []

# 接收line2D对象
line, = plt.plot(xdata, ydata, 'b')
xdata=[i/100 for i in range(0,100)]

# 定义更新函数
def update(frames):
    x_1 = torch.tensor([[i/100] for i in range(0,100)])
    x_11 = torch.cat((torch.full([100, 1],frames), x_1), dim=1)
    ysolve1=net(x_11)
    ysolve=ysolve1.detach().numpy()
    ydata=[ysolve]
    # print(frame_ID)
    line.set_data(xdata, ydata)
    return line,


def init_figure():
    ax.set_xlim(0, 1)
    ax.set_ylim(-1, 1)
    ax.set_xlabel('x')
    ax.set_ylabel('u')


# 调用生成动画的函数生成动图
ani = animation.FuncAnimation(
    fig=fig,
    func=update,
    frames=np.linspace(0, 1, 100),    # [1, 2, 3]
    init_func=init_figure,
    interval=1,   #  每隔多少时间生成一帧图像，单位是ms
    repeat=True,   # 设置不重复，但是还是重复较好
)

plt.show()   # 如果要保存视频和gif就不要show()

ani.save('ani_2.gif', writer='pillow')
#ani.save('ani_1.mp4', writer='ffmpeg')  # 注意，pillow现在似乎不能报错为mp4格式了，可以使用ffmpeg

## 第三组参数

In [71]:
net = zsrDGM_net(6, 200)
v=0.02
alpha=0.9
a=0.9
b=0.8
N1=2
N2=1
N3=6
equation = burgers_PDE(net,v,alpha,a,b)
train = Train(net, equation, BATCH_SIZE=2**8)
train.train(epoch=2000, lr=0.0001)
torch.save(net, 'model3.pkl')
errors = train.get_errors()

Epoch 99 - lr 0.0001 -  loss: 2.132560610771179
Epoch 199 - lr 0.0001 -  loss: 1.36640456199646
Epoch 299 - lr 0.0001 -  loss: 0.21202772927703337
Epoch 399 - lr 0.0001 -  loss: 0.0017142704094294458
Epoch 499 - lr 0.0001 -  loss: 0.0011222312948666513
Epoch 599 - lr 0.0001 -  loss: 0.0011010672460542992
Epoch 699 - lr 0.0001 -  loss: 0.0013381043693516403
Epoch 799 - lr 0.0001 -  loss: 0.00232218072633259
Epoch 899 - lr 0.0001 -  loss: 0.0015043625165708362
Epoch 999 - lr 0.0001 -  loss: 0.0015044907387346028
Epoch 1099 - lr 0.0001 -  loss: 0.0020706460619112476
Epoch 1199 - lr 0.0001 -  loss: 0.0028290474927052858
Epoch 1299 - lr 0.0001 -  loss: 0.0010142992355395108
Epoch 1399 - lr 0.0001 -  loss: 0.0014972515229601413
Epoch 1499 - lr 0.0001 -  loss: 0.0014645669067976997
Epoch 1599 - lr 0.0001 -  loss: 0.002523560514673591
Epoch 1699 - lr 0.0001 -  loss: 0.0009500779217341915
Epoch 1799 - lr 0.0001 -  loss: 0.0018116827687481418
Epoch 1899 - lr 0.0001 -  loss: 0.001302789191249758


In [73]:
fig = plt.figure()
plt.plot(np.log(errors), '-b', label='Errors')
plt.title('Training Loss', fontsize=10)
plt.savefig('err3.jpg')
plt.close(fig)

In [76]:
net=torch.load('model3.pkl')
x_1 = torch.tensor([[i/1000] for i in range(0,1000)])
x_11 = torch.cat((torch.full([1000, 1],1), x_1), dim=1)
ysolve=net(x_11)
ysolve=ysolve.detach().numpy()
x_range=[i/1000 for i in range(0,1000)]
fig=plt.figure(dpi=60)
plt.plot(x_range,(ysolve))
plt.xlim(0,1)
plt.ylim(0.8,0.92)
my_x_ticks = np.arange(0, 1.01, 0.2)
my_y_ticks = np.arange(0.8, 0.93, 0.02)
plt.yticks(my_y_ticks)
plt.xticks(my_x_ticks)
plt.xlabel('x')
plt.ylabel('u')
plt.savefig('3.jpg')
plt.show()

In [77]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
net=torch.load('model3.pkl')
fig, ax = plt.subplots()

# 定义存储数据的列表
xdata = []
ydata = []

# 接收line2D对象
line, = plt.plot(xdata, ydata, 'b')
xdata=[i/100 for i in range(0,100)]

# 定义更新函数
def update(frames):
    x_1 = torch.tensor([[i/100] for i in range(0,100)])
    x_11 = torch.cat((torch.full([100, 1],frames), x_1), dim=1)
    ysolve1=net(x_11)
    ysolve=ysolve1.detach().numpy()
    ydata=[ysolve]
    # print(frame_ID)
    line.set_data(xdata, ydata)
    return line,


def init_figure():
    ax.set_xlim(0, 1)
    ax.set_ylim(0.6,1)
    ax.set_xlabel('x')
    ax.set_ylabel('u')


# 调用生成动画的函数生成动图
ani = animation.FuncAnimation(
    fig=fig,
    func=update,
    frames=np.linspace(0, 1, 100),    # [1, 2, 3]
    init_func=init_figure,
    interval=1,   #  每隔多少时间生成一帧图像，单位是ms
    repeat=True,   # 设置不重复，但是还是重复较好
)

plt.show()   # 如果要保存视频和gif就不要show()

ani.save('ani_3.gif', writer='pillow')
#ani.save('ani_1.mp4', writer='ffmpeg')  # 注意，pillow现在似乎不能报错为mp4格式了，可以使用ffmpeg

## 第四个参数


In [78]:
net = zsrDGM_net(6, 200)
v=0.01
alpha=0.75
a=0.9
b=0.1
N1=1
N2=1
N3=3
equation = burgers_PDE(net,v,alpha,a,b)
train = Train(net, equation, BATCH_SIZE=2**8)
train.train(epoch=2000, lr=0.0001)
torch.save(net, 'model4.pkl')
errors = train.get_errors()

Epoch 99 - lr 0.0001 -  loss: 1.0504060238599777
Epoch 199 - lr 0.0001 -  loss: 0.5562401777505874
Epoch 299 - lr 0.0001 -  loss: 0.019698284538462757
Epoch 399 - lr 0.0001 -  loss: 0.007384422332979739
Epoch 499 - lr 0.0001 -  loss: 0.004216504744254053
Epoch 599 - lr 0.0001 -  loss: 0.002959045625757426
Epoch 699 - lr 0.0001 -  loss: 0.002658561037387699
Epoch 799 - lr 0.0001 -  loss: 0.0023841389268636705
Epoch 899 - lr 0.0001 -  loss: 0.0019270122901070862
Epoch 999 - lr 0.0001 -  loss: 0.0018809000519104302
Epoch 1099 - lr 0.0001 -  loss: 0.0016562186880037188
Epoch 1199 - lr 0.0001 -  loss: 0.001701307010371238
Epoch 1299 - lr 0.0001 -  loss: 0.0012053268740419298
Epoch 1399 - lr 0.0001 -  loss: 0.0014758457412244752
Epoch 1499 - lr 0.0001 -  loss: 0.0011242558166850358
Epoch 1599 - lr 0.0001 -  loss: 0.0012271988461725414
Epoch 1699 - lr 0.0001 -  loss: 0.0013148327043745666
Epoch 1799 - lr 0.0001 -  loss: 0.0010198574746027589
Epoch 1899 - lr 0.0001 -  loss: 0.00108823089161887

In [81]:
fig = plt.figure()
plt.plot(np.log(errors), '-b', label='Errors')
plt.title('Training Loss', fontsize=10)
plt.savefig('err4.jpg')
plt.close(fig)

In [80]:
net=torch.load('model4.pkl')
x_1 = torch.tensor([[i/1000] for i in range(0,1000)])
x_11 = torch.cat((torch.full([1000, 1],1), x_1), dim=1)
ysolve=net(x_11)
ysolve=ysolve.detach().numpy()
x_range=[i/1000 for i in range(0,1000)]
fig=plt.figure(dpi=60)
plt.plot(x_range,(ysolve))
plt.xlim(0,1)
plt.ylim(0,1)
my_x_ticks = np.arange(0, 1.01, 0.2)
my_y_ticks = np.arange(0, 1, 0.2)
plt.yticks(my_y_ticks)
plt.xticks(my_x_ticks)
plt.xlabel('x')
plt.ylabel('u')
plt.savefig('4.jpg')
plt.show()

In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
net=torch.load('model4.pkl')
fig, ax = plt.subplots()

# 定义存储数据的列表
xdata = []
ydata = []

# 接收line2D对象
line, = plt.plot(xdata, ydata, 'b')
xdata=[i/100 for i in range(0,100)]

# 定义更新函数
def update(frames):
    x_1 = torch.tensor([[i/100] for i in range(0,100)])
    x_11 = torch.cat((torch.full([100, 1],frames), x_1), dim=1)
    ysolve1=net(x_11)
    ysolve=ysolve1.detach().numpy()
    ydata=[ysolve]
    # print(frame_ID)
    line.set_data(xdata, ydata)
    return line,


def init_figure():
    ax.set_xlim(0, 1)
    ax.set_ylim(0,1)
    ax.set_xlabel('x')
    ax.set_ylabel('u')


# 调用生成动画的函数生成动图
ani = animation.FuncAnimation(
    fig=fig,
    func=update,
    frames=np.linspace(0, 1, 100),    # [1, 2, 3]
    init_func=init_figure,
    interval=1,   #  每隔多少时间生成一帧图像，单位是ms
    repeat=True,   # 设置不重复，但是还是重复较好
)

plt.show()   # 如果要保存视频和gif就不要show()

ani.save('ani_4.gif', writer='pillow')
#ani.save('ani_1.mp4', writer='ffmpeg')  # 注意，pillow现在似乎不能报错为mp4格式了，可以使用ffmpeg

NameError: name 'torch' is not defined